# Glacier grids from RGI:

Creates monthly grid files for the MBM to make PMB predictions over the whole glacier grid. The files come from the RGI grid with OGGM topography. Computing takes a long time because of the conversion to monthly format.
## Setting up:

In [ ]:
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
import re
import massbalancemachine as mbm
import geopandas as gpd
import matplotlib.pyplot as plt
from cmcrameri import cm
from oggm import utils, workflow
from oggm import cfg as oggmCfg
import geopandas as gpd
import geopandas as gpd
import traceback
import salem
import oggm


# scripts
from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.geodata import *
from scripts.xgb_helpers import *
from scripts.config_CH import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

cfg = mbm.SwitzerlandConfig()

In [ ]:
seed_all(cfg.seed)
free_up_cuda()  # in case no memory

# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)

# Climate columns
vois_climate = [
    't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
]
# Topographical columns
voi_topographical = [
    "aspect",
    "slope",
    "hugonnet_dhdt",
    "consensus_ice_thickness",
    "millan_v",
    "topo",
]


In [ ]:
gdirs, rgidf = initialize_oggm_glacier_directories(
    cfg,
    rgi_region="11",
    rgi_version="6",
    base_url=
    "https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5_w_data/",
    log_level='WARNING',
    task_list=None,
)
# Save OGGM xr for all needed glaciers in RGI region 11.6:
export_oggm_grids(cfg, gdirs)

In [ ]:
# RGI Ids:
# Read glacier ids:
rgi_df = pd.read_csv(cfg.dataPath + path_glacier_ids, sep=',')
rgi_df.rename(columns=lambda x: x.strip(), inplace=True)
rgi_df.sort_values(by='short_name', inplace=True)
rgi_df.set_index('short_name', inplace=True)
rgi_df.loc['rhone']

## Create RGI grids for all glaciers:

In [ ]:
path_RGIs = cfg.dataPath + path_OGGM + 'xr_grids/'
glaciers = os.listdir(path_RGIs)

print(f"Found {len(glaciers)} glaciers in RGI region 11.6")

# Open an example
# rgi_gl = gdirs[0].rgi_id
rgi_gl = 'RGI60-11.01238'

ds = xr.open_dataset(path_RGIs + rgi_gl + '.zarr')
glacier_mask = np.where(ds['glacier_mask'].values == 0, np.nan,
                        ds['glacier_mask'].values)

# Create glacier mask
ds = ds.assign(masked_slope=glacier_mask * ds['slope'])
ds = ds.assign(masked_elev=glacier_mask * ds['topo'])
ds = ds.assign(masked_aspect=glacier_mask * ds['aspect'])
ds = ds.assign(masked_dis=glacier_mask * ds['dis_from_border'])

# Assign other variables only if available
if 'hugonnet_dhdt' in ds:
    ds = ds.assign(masked_hug=glacier_mask * ds['hugonnet_dhdt'])
if 'consensus_ice_thickness' in ds:
    ds = ds.assign(masked_cit=glacier_mask * ds['consensus_ice_thickness'])
if 'millan_v' in ds:
    ds = ds.assign(masked_miv=glacier_mask * ds['millan_v'])

glacier_indices = np.where(ds['glacier_mask'].values == 1)

fig, axs = plt.subplots(1, 4, figsize=(16, 8), sharey=True)

ds.masked_aspect.plot(ax=axs[0], cmap='twilight_shifted', add_colorbar=False)
ds.masked_slope.plot(ax=axs[1], cmap='cividis', add_colorbar=False)
ds.masked_elev.plot(ax=axs[2], cmap='terrain', add_colorbar=False)
ds.glacier_mask.plot(ax=axs[3], cmap='binary', add_colorbar=False)

axs[0].set_title("Aspect OGGM")
axs[1].set_title("Slope OGGM")
axs[2].set_title("DEM OGGM")
axs[3].set_title("Glacier mask OGGM")

In [ ]:
def create_glacier_grid_RGI(ds: xr.Dataset, years: list,
                            glacier_indices: "tuple[np.array, np.array]",
                            gdir: oggm.GlacierDirectory, rgi_gl: str):

    # Assuming the coordinate variables are named 'x' and 'y' in your dataset
    x_coords = ds['x'].values
    y_coords = ds['y'].values

    # Retrieve the x and y values using the glacier indices
    glacier_x_vals = x_coords[glacier_indices[1]]
    glacier_y_vals = y_coords[glacier_indices[0]]

    # Convert glacier coordinates to latitude and longitude
    # Transform stake coord to glacier system:
    transf = pyproj.Transformer.from_proj(gdir.grid.proj,
                                          salem.wgs84,
                                          always_xy=True)
    lon, lat = transf.transform(glacier_x_vals, glacier_y_vals)

    # Glacier mask as boolean array:
    gl_mask_bool = ds['glacier_mask'].values.astype(bool)

    # Create a DataFrame
    data_grid = {
        'RGIId': [rgi_gl] * len(ds.masked_elev.values[gl_mask_bool]),
        'POINT_LAT': lat,
        'POINT_LON': lon,
        'aspect': ds.masked_aspect.values[gl_mask_bool],
        'slope': ds.masked_slope.values[gl_mask_bool],
        'topo': ds.masked_elev.values[gl_mask_bool],
    }

    # add other variables if available
    if 'masked_hug' in ds:
        data_grid['hugonnet_dhdt'] = ds.masked_hug.values[gl_mask_bool]
    if 'masked_cit' in ds:
        data_grid['consensus_ice_thickness'] = ds.masked_cit.values[
            gl_mask_bool]
    if 'masked_miv' in ds:
        data_grid['millan_v'] = ds.masked_miv.values[gl_mask_bool]

    df_grid = pd.DataFrame(data_grid)

    # Match to WGMS format:
    df_grid['POINT_ID'] = np.arange(1, len(df_grid) + 1)
    df_grid['N_MONTHS'] = 12
    df_grid['POINT_ELEVATION'] = df_grid[
        'topo']  # no other elevation available
    df_grid['POINT_BALANCE'] = 0  # fake PMB for simplicity (not used)
    num_rows_per_year = len(df_grid)
    # Repeat the DataFrame num_years times
    df_grid = pd.concat([df_grid] * len(years), ignore_index=True)
    # Add the 'year' and date columns to the DataFrame
    df_grid['YEAR'] = np.repeat(
        years, num_rows_per_year
    )  # 'year' column that has len(df_grid) instances of year
    df_grid['FROM_DATE'] = df_grid['YEAR'].apply(lambda x: str(x) + '1001')
    df_grid['TO_DATE'] = df_grid['YEAR'].apply(lambda x: str(x + 1) + '0930')
    df_grid["PERIOD"] = "annual"

    return df_grid

In [ ]:

def create_masked_glacier(path_RGIs, rgi_gl):
    # Load dataset
    ds = xr.open_dataset(path_RGIs + rgi_gl + '.zarr')

    # Check if 'glacier_mask' exists
    if 'glacier_mask' not in ds:
        raise ValueError(f"'glacier_mask' variable not found in dataset {rgi_gl}")

    # Create glacier mask
    glacier_mask = np.where(ds['glacier_mask'].values == 0, np.nan,
                            ds['glacier_mask'].values)

    # Apply mask to core variables
    ds = ds.assign(masked_slope=glacier_mask * ds['slope'])
    ds = ds.assign(masked_elev=glacier_mask * ds['topo'])
    ds = ds.assign(masked_aspect=glacier_mask * ds['aspect'])
    ds = ds.assign(masked_dis=glacier_mask * ds['dis_from_border'])

    # Apply mask to optional variables if present
    if 'hugonnet_dhdt' in ds:
        ds = ds.assign(masked_hug=glacier_mask * ds['hugonnet_dhdt'])
    if 'consensus_ice_thickness' in ds:
        ds = ds.assign(masked_cit=glacier_mask * ds['consensus_ice_thickness'])
    if 'millan_v' in ds:
        ds = ds.assign(masked_miv=glacier_mask * ds['millan_v'])

    # Indices where glacier_mask == 1
    glacier_indices = np.where(ds['glacier_mask'].values == 1)

    return ds, glacier_indices

### Create masked grids:

In [ ]:
path_xr_grids = os.path.join(cfg.dataPath, 'GLAMOS/topo/RGI_v6_11/', 'xr_masked_grids/')

RUN = False
if RUN:
    emptyfolder(path_xr_grids)

    for gdir in tqdm(gdirs):
        rgi_gl = gdir.rgi_id

        try:
            # Create masked glacier dataset
            ds, glacier_indices = create_masked_glacier(path_RGIs, rgi_gl)
        except ValueError as e:
            print(f"Skipping {rgi_gl}: {e}")
            continue  # Skip to next glacier

        dx_m, dy_m = get_res_from_projected(ds)

        # Coarsen to 50 m resolution if needed
        if 20 < dx_m < 50:
            ds = coarsenDS_mercator(ds, target_res_m=50)
            dx_m, dy_m = get_res_from_projected(ds)

        # Save xarray dataset
        save_path = os.path.join(path_xr_grids, f"{rgi_gl}.zarr")
        ds.to_zarr(save_path)

### Create monthly dataframes:

In [ ]:
for gdir in gdirs:
    if gdir.rgi_id == 'RGI60-11.01238':
        gdir_rhone = gdir
        
years = range(2000, 2024)

path_rgi_alps = os.path.join(cfg.dataPath, 'GLAMOS/topo/gridded_topo_inputs/RGI_v6_11/')
os.makedirs(path_rgi_alps, exist_ok=True)
emptyfolder(path_rgi_alps)

valid_rgis = [f.replace('.zarr', '') for f in os.listdir(path_xr_grids) if f.endswith('.zarr')]

for gdir in tqdm(gdirs, desc="Processing glaciers"):
    rgi_gl = gdir.rgi_id

    if rgi_gl not in valid_rgis:
        print(f"Skipping {rgi_gl}: not found in valid RGI glaciers")
        continue

    try:
        file_path = os.path.join(path_xr_grids, f"{rgi_gl}.zarr")
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Missing file: {file_path}")

        ds = xr.open_dataset(file_path)

        # Get glacier_indices safely
        if 'glacier_mask' in ds:
            glacier_indices = np.where(ds['glacier_mask'].values == 1)
        else:
            raise ValueError(f"Missing 'glacier_mask' in dataset for {rgi_gl}")

        # Create glacier grid
        try:
            df_grid = create_glacier_grid_RGI(ds, years, glacier_indices, gdir, rgi_gl)
        except Exception as e:
            print(f"Failed creating glacier grid for {rgi_gl}: {e}")
            continue

        df_grid.reset_index(drop=True, inplace=True)

        # Add GLWD_ID
        df_grid['GLWD_ID'] = df_grid.apply(
            lambda x: mbm.data_processing.utils.get_hash(f"{x.RGIId}_{x.YEAR}"), axis=1)
        df_grid['GLWD_ID'] = df_grid['GLWD_ID'].astype(str)
        df_grid['GLACIER'] = df_grid['RGIId']

        # Wrap Dataset creation & climate feature extraction
        try:
            dataset_grid = mbm.data_processing.Dataset(
                cfg=cfg,
                data=df_grid,
                region_name='CH',
                data_path=os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv)
            )

            era5_climate_data = os.path.join(cfg.dataPath, path_ERA5_raw, 'era5_monthly_averaged_data_Alps.nc')
            geopotential_data = os.path.join(cfg.dataPath, path_ERA5_raw, 'era5_geopotential_pressure_Alps.nc')

            dataset_grid.get_climate_features(
                climate_data=era5_climate_data,
                geopotential_data=geopotential_data,
                change_units=True,
                smoothing_vois={
                    'vois_climate': vois_climate,
                    'vois_other': ['ALTITUDE_CLIMATE']
                })
        except Exception as e:
            print(f"Failed adding climate features for {rgi_gl}: {e}")
            continue

        # Prepare output folder
        folder_path = os.path.join(path_rgi_alps, rgi_gl)
        os.makedirs(folder_path, exist_ok=True)

        # Process each year
        for year in years:
            try:
                df_grid_y = dataset_grid.data[dataset_grid.data.YEAR == year].copy()

                voi_topographical_sub = [voi for voi in voi_topographical if voi in df_grid_y.columns]

                dataset_grid_yearly = mbm.data_processing.Dataset(
                    cfg=cfg,
                    data=df_grid_y,
                    region_name='CH',
                    data_path=os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv)
                )

                dataset_grid_yearly.convert_to_monthly(
                    meta_data_columns=cfg.metaData,
                    vois_climate=vois_climate,
                    vois_topographical=voi_topographical_sub
                )

                save_path = os.path.join(folder_path, f"{rgi_gl}_grid_{year}.parquet")
                dataset_grid_yearly.data.to_parquet(save_path, engine="pyarrow", compression="snappy")
                print(f"Saved: {save_path}")

            except Exception as e:
                print(f"Failed processing {rgi_gl} for year {year}: {e}")
                continue

    except Exception as e:
        print(f"Error with glacier {rgi_gl}: {e}")
        continue

In [ ]:
for gdir in gdirs:
    if gdir.rgi_id == 'RGI60-11.01238':
        gdir_rhone = gdir

# Look at one example
# load the dataset
year = 2000
rgi_gl = gdir_rhone.rgi_id

df = pd.read_parquet(
    os.path.join(path_rgi_alps, rgi_gl, f"{rgi_gl}_grid_{year}.parquet"))
df = df[df.MONTHS == 'sep']
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
voi = [
    't2m', 'tp', 'ALTITUDE_CLIMATE', 'ELEVATION_DIFFERENCE', 'hugonnet_dhdt',
    'consensus_ice_thickness'
]
axs = axs.flatten()
for i, var in enumerate(voi):
    sns.scatterplot(df,
                    x='POINT_LON',
                    y='POINT_LAT',
                    hue=var,
                    s=5,
                    alpha=0.5,
                    palette='twilight_shifted',
                    ax=axs[i])